Cryptocurrency trading bot.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_base.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger base class.

# Library imports.
from cryptocurrency.resample import resample
from binance.client import Client
from abc import abstractmethod, ABC
from time import sleep, time
from os.path import exists, join
from os import mkdir

import pandas as pd

class Crypto_logger_base(ABC):
    def __init__(self, interval='15s', delay=4.7, buffer_size=3000, directory='crypto_logs', 
                 log_name='crypto_log', raw=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param directory: the directory where to output the logs.
        :param log_name: name of the log file.
        :param raw: whether the log dumps raw (instantaneous) or OHLCV data.
        """
        self.interval = interval
        self.delay = delay
        self.buffer_size = buffer_size
        self.directory = directory
        self.raw = raw

        self.log_name = join(self.directory, log_name + '.txt')
        self.log_screened_name = join(self.directory, log_name + '_screened.txt')

        if not exists(self.directory):
            mkdir(self.directory)

    #self.get_from_file(log_name=self.log_name, from_raw=False)
    #self.get_from_file(log_name=self.input_log_name, from_raw=self.load_from_ohlcv)
    def get_from_file(self, log_name, from_raw=False):
        if from_raw:
            dataset = pd.read_csv(log_name, header=0, index_col=0)
        else:
            dataset = pd.read_csv(log_name, header=[0, 1], index_col=0)
        dataset.index = pd.DatetimeIndex(dataset.index)
        return dataset.sort_index(axis='index')

    @abstractmethod
    def get(self, **kwargs):
        raise NotImplementedError()

    @abstractmethod
    def screen(self, **kwargs):
        raise NotImplementedError()

    def put(self, dataset):
        dataset = dataset.copy().reset_index()
        if self.raw:
            dataset = dataset.drop_duplicates(subset=['symbol', 'count'], 
                                              keep='first', ignore_index=True)
        else:
            dataset = dataset.drop_duplicates(keep='last', ignore_index=True)

        if 'date' in dataset.columns:
            min_index_int = dataset[dataset['date'] == self.min_index].index[0]
            dataset = dataset.set_index('date')
        if not self.raw:
            dataset = resample(dataset, self.interval)
        if 'date' in dataset.columns:
            dataset = dataset.iloc[min_index_int:]

        dataset = dataset.tail(self.buffer_size)
        dataset.to_csv(self.log_name)
        self.min_index = dataset.index[0]
        return dataset

    def start(self, append=False, roll=0):
        """Main logger loop."""
        print('Starting crypto logger.')

        if exists(self.log_name) and 'output' in self.log_name:
            self.dataset = self.get_from_file(log_name=self.log_name, from_raw=False)
            self.dataset = self.dataset.tail(self.buffer_size)
        else:
            self.dataset = self.get()

        self.min_index = self.dataset.index[-1]
        self.dataset = self.put(self.dataset)

        while True:
            try:
                dataset = pd.concat([self.dataset, self.get()], axis='index', join='outer')
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                self.dataset = self.put(dataset)
            except (KeyboardInterrupt, SystemExit):
                print('Saving latest complete dataset...')
                self.dataset = self.put(dataset)
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                dataset_screened = self.screen(self.dataset)
                if dataset_screened is not None:
                    if roll != 0:
                        if append and exists(self.log_screened_name):
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    elif append:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    else:
                        dataset_screened.to_csv(self.log_screened_name)
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            sleep(self.delay)
        print('Crypto logger process done.')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_output.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for arbitrary intervals.

# Library imports.
#from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.indicators import filter_in_market, screen_one
from os.path import exists, join

import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_output(Crypto_logger_base):
    def __init__(self, delay=10, interval_input='15s', interval='15s', buffer_size=100, 
                 input_log_name='input'):
        """
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param interval_input: OHLCV interval from input log. Default is 15 seconds.
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param input_log_name: the directory where to take the logs from.
        """
        self.data_before = pd.DataFrame()
        input_log_name = 'crypto_' + input_log_name + '_log_'
        self.load_from_ohlcv = interval_input != interval
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_output_log_' + interval, 
                         raw=False)

        self.input_log_name = \
            join(self.directory, input_log_name + interval_input + '.txt')
        self.input_log_screened_name = \
            join(self.directory, input_log_name + interval_input + '_screened.txt')

    def screen(self, dataset):
        if exists(self.input_log_screened_name):
            input_filtered = pd.read_csv(self.input_log_screened_name, header=0, index_col=0)
            input_filter = set(input_filtered['symbol'].tolist())
            old_columns = set(dataset.columns.get_level_values(0).tolist())
            new_columns = list(input_filter & old_columns)
            dataset = dataset[new_columns]
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            #dataset = dataset.rename(columns={'base_volume': 'volume'})
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            assets = filter_in_market(screen_one, dataset)
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            #dataset = dataset.rename(columns={'volume': 'base_volume'})
            #dataset.columns = dataset.columns.swaplevel(0, 1)
            return input_filtered[input_filtered['symbol'].isin(assets)]
        else:
            return None

    def resample_from_raw(self, df):
        df = df[['symbol', 'close', 'rolling_base_volume', 'rolling_quote_volume']]
        df['base_volume'] = df['rolling_base_volume'].copy()
        df['quote_volume'] = df['rolling_quote_volume'].copy()
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['close', 'rolling_base_volume', 
                                    'rolling_quote_volume', 
                                    'base_volume', 'quote_volume'], 
                            aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                     'base_volume': 'max', 'quote_volume': 'max', 
                                     'rolling_base_volume': 'max', 
                                     'rolling_quote_volume': 'max'})
        df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                               names=('pair', 'symbol'))
        df = df.rename(columns={'close_first': 'open', 'close_max': 'high', 
                                'close_min': 'low', 'close_last': 'close', 
                                'base_volume_max': 'base_volume', 
                                'quote_volume_max': 'quote_volume', 
                                'rolling_base_volume_max': 'rolling_base_volume', 
                                'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                       level=0)
        df['base_volume'] = df['base_volume'].fillna(method='pad')
        df['base_volume'].iloc[0] = 0
        df['quote_volume'] = df['quote_volume'].fillna(method='pad')
        df['quote_volume'].iloc[0] = 0
        df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
        df['rolling_base_volume'].iloc[0] = 0
        df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
        df['rolling_quote_volume'].iloc[0] = 0
        df = df.sort_index().iloc[1:]
        df.columns = df.columns.swaplevel(0, 1)
        return df

    def get(self):
        dataset = self.get_from_file(log_name=self.input_log_name, 
                                     from_raw=not self.load_from_ohlcv)
        if not self.load_from_ohlcv:
            dataset = self.resample_from_raw(dataset)
        return dataset.tail(2)

Main.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_output_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_output import Crypto_logger_output

crypto_logger_output_1min = Crypto_logger_output(delay=44, 
                                                 interval_input='15s', 
                                                 interval='1min', 
                                                 buffer_size=3000, 
                                                 input_log_name='output')
crypto_logger_output_1min.start(append=False, roll=1000)

In [ ]:
import pandas as pd
input_log_name = 'crypto_logs/crypto_input_log_15s.txt'
df = pd.read_csv(input_log_name, header=0, index_col=0)
df.index = pd.DatetimeIndex(df.index)
df = df.sort_index(axis='index')
#df = crypto_logger_output_1min.resample_from_raw(df)
df

In [ ]:
import pandas as pd
#crypto_output_log_1d = crypto_logger_output_1d.log_name
crypto_output_log_1d = 'crypto_logs/crypto_output_log_1d.txt'
df_1d = pd.read_csv(crypto_output_log_1d, header=[0, 1], index_col=0)
df_1d.index = pd.DatetimeIndex(df_1d.index)
df_1d = df_1d.sort_index(axis='index')
df_1d['BTC'].tail(20)

In [ ]:
import pandas as pd
#crypto_output_log_1h = crypto_logger_output_1h.log_name
crypto_output_log_1h = 'crypto_logs/crypto_output_log_1h.txt'
df_1h = pd.read_csv(crypto_output_log_1h, header=[0, 1], index_col=0)
df_1h.index = pd.DatetimeIndex(df_1h.index)
df_1h = df_1h.sort_index(axis='index')
df_1h['BTC'].tail(20)

In [ ]:
import pandas as pd
#crypto_output_log_1min = crypto_logger_output_1min.log_name
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min = df_1min.sort_index(axis='index')
df_1min['BTC'].tail(20)

In [ ]:
dataset = crypto_logger_output_1min.dataset
dataset

In [ ]:
dataset['BTCUSDT']

In [ ]:
dataset['ZILBUSD']

In [ ]:
import pandas_ta as ta

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_relative_volume_levels_smoothed_trigger(data, average1=26, average2=14):
    volume = data['volume']
    volume_average = ta.sma(close=volume, length=average1)
    relative_volume = volume / volume_average
    smoothed_relative_volume = ta.sma(close=relative_volume, length=average2)
    return (smoothed_relative_volume > threshold).iat[-1]

def get_relative_volume_levels_at_time_smoothed_thresholded(data):
    try:
        volume = data['volume']
        #volume = volume.groupby(pd.Grouper(freq='D')).cumsum()
        cum_volume = volume.groupby(pd.Grouper(freq='24h')).cumsum()
        #volume = volume.groupby(pd.Grouper(freq='60m')).cumsum()
        cum_rvol = (cum_volume / cum_volume.shift(1)).fillna(method='pad')
        rvol = (volume / volume.shift(1)).fillna(method='pad')
        bar_up = (data['close'] > data['open'])
        bar_up |= (data['close'] == data['open']) & (data['close'].diff() > 0)
        bar_up = bar_up.astype(int)
        bar_up = bar_up * 2 - 1
        #rvol *= bar_up
        cum_rvol_dir = cum_rvol * bar_up
        rvol_dir = rvol * bar_up
        rvol_indicator = ta.hma(rvol, length=14, talib=True)
        rvol_dir_indicator = ta.hma(rvol_dir, length=14, talib=True)
        cum_rvol_indicator = ta.hma(cum_rvol, length=14, talib=True)
        cum_rvol_dir_indicator = ta.hma(cum_rvol_dir, length=14, talib=True)
        rvol_indicator = rvol_indicator.rename('relative_volume_levels_smoothed')
        rvol_dir_indicator = rvol_dir_indicator.rename('relative_volume_levels_dir_smoothed')
        cum_rvol_indicator = cum_rvol_indicator.rename('cum_relative_volume_levels_smoothed')
        cum_rvol_dir_indicator = cum_rvol_dir_indicator.rename('cum_relative_volume_levels_dir_smoothed')
        #threshold = (ta.sma(rvol, length=100, talib=True) + ta.stdev(rvol, length=100, talib=True))
        threshold_dir = 0
        threshold = 2
        rvol_thresholded = (rvol_indicator > threshold).iat[-1]
        rvol_dir_thresholded = (rvol_dir_indicator > threshold_dir).iat[-1]
        cum_rvol_thresholded = (cum_rvol_indicator > threshold).iat[-1]
        cum_rvol_dir_thresholded = (cum_rvol_dir_indicator > threshold_dir).iat[-1]
        trigger = (rvol_thresholded | rvol_dir_thresholded | cum_rvol_thresholded | cum_rvol_dir_thresholded)
    except Exception as e:
        print('rvol exception:', e)
        trigger = False
    return trigger

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iat[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iat[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iat[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iat[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iat[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iat[-2]) or \
            (MACD['MACD_12_26_9'].iat[-1] > MACD['MACDs_12_26_9'].iat[-1]))

def get_not_square_wave_triggers(data, multiplier_schedule):
    triggers = True
    for multiplier in multiplier_schedule:
        period_1 = -4 * multiplier
        uniques_1 = 2 * multiplier
        square_wave_trigger_1 = (data.iloc[period_1:]['close'].unique().size < uniques_1)
        if square_wave_trigger_1:
            triggers = False
            break
        else:
            period_2 = -15 * multiplier
            uniques_2 = 6 * multiplier
            square_wave_trigger_2 = (data.iloc[period_2:]['close'].unique().size < uniques_2)
            if square_wave_trigger_2:
                triggers = False
                break
    return triggers

def get_minute_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1, 2, 3, 5, 10, 15, 20, 45])

def get_hourly_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1, 2, 3, 4, 6, 8, 12])

def get_daily_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_volume_minimum_trigger(data):
    return (data['volume'] > 1000000).iat[-1]

def get_daily_volume_change_trigger(data):
    return ((data['volume'].pct_change(1) * 100) > 300).iat[-1]

def get_minute_daily_volume_minimum_trigger(data):
    return (data['rolling_base_volume'] > 1000000).iat[-1]

def get_minute_daily_volume_change_trigger(data):
    return ((data['rolling_base_volume'].pct_change(1440) * 100) > 300).iat[-1]

def get_rising_volume_trigger(data):
    return (data['rolling_base_volume'].diff(1) > 0).iat[-1]

In [ ]:
df = df_1d.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_daily_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_relative_volume_levels_smoothed_trigger, df).tolist())
filtered_1d = pd.Series(list(filtered_1 & filtered_2 & filtered_3))
filtered_1d

In [ ]:
df = df_1h.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_hourly_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_relative_volume_levels_at_time_smoothed_thresholded, df).tolist())
filtered_1h = pd.Series(list(filtered_1 & filtered_2))
filtered_1h

In [ ]:
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_minute_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_minute_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_minute_daily_volume_change_trigger, df).tolist())
filtered_4 = set(filter_in_market(get_rising_volume_trigger, df).tolist())
filtered_1min = pd.Series(list(filtered_1 & filtered_2 & filtered_3 & filtered_4))
filtered_1min

In [ ]:
list(set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#conversion_table.sort_values(by='rolling_traded_volume', ascending=False).reset_index(drop=True).head(50)

In [ ]:
from cryptocurrency.conversion import convert_price, get_base_asset_from_pair, get_quote_asset_from_pair
from pandas import concat, DataFrame
import time

def get_conversion_table(client, exchange_info, as_pair=False):
    """
    Fetches and prepares data used to calculate prices, volumes and other stats.
    :param client: object from python-binance useful for calling client.get_ticker().
    :param exchange_info: Pre-calculated exchange information on all tickers.
    :return: pd.DataFrame containing all preprocessed conversion table info.
    :column is_shorted: is the symbol made from inversion.
    :column symbol: concatenated string made from base_asset and quote_asset.
    :column shorted_symbol: symbol with inverted base_asset and quote_asset.
    :column base_asset: asset on the left.
    :column quote_asset: asset on the right.
    :column price_change: (close - open).
    :column price_change_percent: (((close - open) / open) * 100).
    :column USDT_price_change: (USDT_close - USDT_open).
    :column USDT_price_change_percent: (((USDT_close - USDT_open) / USDT_open) * 100).
    :column weighted_average_price: weighted average price.
    :column close_shifted: close price of the previous day.
    :column open: open price of the day.
    :column high: high price of the day.
    :column low: low price of the day.
    :column close: close price of the day.
    :column last_volume: volume of the last price update.
    :column USDT_bid_price: USDT-converted bid price.
    :column USDT_ask_price: USDT-converted ask price.
    :column USDT_bid_volume: USDT-converted bid volume.
    :column USDT_ask_volume: USDT-converted ask volume.
    :column bid_price: price of the bid.
    :column bid_volume: volume of the bid at bid_price.
    :column ask_price: price of the ask.
    :column ask_volume: volume of the ask at ask_price.
    :column rolling_base_volume: rolling_base_volume given by the API.
    :column rolling_quote_volume: rolling_quote_volume given by the API.
    :column open_time: close_time minus 24 hours.
    :column close_time: time from epoch in milliseconds of the last price update.
    :column first_ID: transaction ID from 1 day ago.
    :column last_ID: latest transaction ID.
    :column count: value calculated by subtracting first_ID from last_ID.
    :column USDT_open: USDT-converted open price.
    :column USDT_high: USDT-converted high price.
    :column USDT_low: USDT-converted low price.
    :column USDT_price: USDT-converted close price.
    :column rolling_USDT_base_volume: USDT-converted rolling_base_volume.
    :column rolling_USDT_quote_volume: USDT-converted rolling_quote_volume.
    :column rolling_traded_volume: sum by base_asset of all USDT-converted volumes.
    :column importance: rolling_USDT_base_volume divided by rolling_traded_volume.
    :column traded_price: sum by base_asset of all (close prices times importance).
    :column traded_bid_price: sum by base_asset of all (bid prices times importance).
    :column traded_ask_price: sum by base_asset of all (ask prices times importance).
    :column bid_ask_change_percent: ((ask_price - bid_price) / ask_price) * 100).
    :column bid_ask_volume_percent: ((bid_volume / (bid_volume + ask_volume)) * 100).
    :column traded_bid_ask_change_percent: ((traded_ask_price - traded_bid_price) / traded_ask_price) * 100).
    :column traded_bid_ask_volume_percent: ((traded_bid_volume / (traded_bid_volume + traded_ask_volume)) * 100).
    """
    conversion_table = DataFrame(client.get_ticker())

    conversion_table = conversion_table[conversion_table['symbol'].isin(exchange_info['symbol'])]
    conversion_table['base_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=exchange_info))
    conversion_table['quote_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_quote_asset_from_pair(x, exchange_info=exchange_info))

    conversion_table = \
        conversion_table.rename(columns={'openPrice': 'open', 'highPrice': 'high', 'lowPrice': 'low', 
                                         'lastPrice': 'close', 'volume': 'rolling_base_volume', 
                                         'quoteVolume': 'rolling_quote_volume', 'lastQty': 'last_volume', 
                                         'bidPrice': 'bid_price', 'askPrice': 'ask_price', 
                                         'bidQty': 'bid_volume', 'askQty': 'ask_volume', 
                                         'firstId': 'first_ID', 'lastId': 'last_ID', 
                                         'openTime': 'open_time', 'closeTime': 'close_time', 
                                         'prevClosePrice': 'close_shifted', 
                                         'weightedAvgPrice': 'weighted_average_price', 
                                         'priceChange': 'price_change', 
                                         'priceChangePercent': 'price_change_percent'})
    conversion_table[['close_time', 'price_change_percent', 'open', 'high', 'low', 'close', 'close_shifted', 
                      'bid_price', 'ask_price', 'bid_volume', 'ask_volume', 'rolling_base_volume', 
                      'rolling_quote_volume', 'count']] = \
        conversion_table[['close_time', 'price_change_percent', 'open', 'high', 'low', 'close', 'close_shifted', 
                          'bid_price', 'ask_price', 'bid_volume', 'ask_volume', 'rolling_base_volume', 
                          'rolling_quote_volume', 'count']].astype(float)
    conversion_table['close_time'] = conversion_table['close_time'].astype(int)

    is_dst = time.localtime().tm_isdst
    timezone = time.tzname[is_dst]
    offset_s = time.altzone if is_dst else time.timezone
    offset = (offset_s / 60 / 60)
    conversion_table['close_time'] = (conversion_table['close_time'] + offset_s * 1000)

    conversion_table['rolling_base_quote_volume'] = \
        conversion_table['rolling_quote_volume'] / conversion_table['close']
    conversion_table['USDT_open'] = \
        conversion_table.apply(lambda x: convert_price(size=1, from_asset=x['base_asset'], to_asset='USDT', 
                                                       conversion_table=conversion_table, 
                                                       exchange_info=exchange_info, key='open'), axis='columns')
    conversion_table['USDT_high'] = \
        conversion_table.apply(lambda x: convert_price(size=1, from_asset=x['base_asset'], to_asset='USDT', 
                                                       conversion_table=conversion_table, 
                                                       exchange_info=exchange_info, key='close'), axis='columns')
    conversion_table['USDT_low'] = \
        conversion_table.apply(lambda x: convert_price(size=1, from_asset=x['base_asset'], to_asset='USDT', 
                                                       conversion_table=conversion_table, 
                                                       exchange_info=exchange_info, key='close'), axis='columns')
    conversion_table['USDT_price'] = \
        conversion_table.apply(lambda x: convert_price(size=1, from_asset=x['base_asset'], to_asset='USDT', 
                                                       conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), axis='columns')
    conversion_table['rolling_USDT_base_volume'] = \
        conversion_table['rolling_base_volume'] * conversion_table['USDT_price']
    conversion_table['rolling_USDT_quote_volume'] = \
        conversion_table['rolling_base_quote_volume'] * conversion_table['USDT_price']

    conversion_table['USDT_bid_price'] = \
        conversion_table.apply(lambda x: convert_price(size=x['bid_price'], from_asset=x['base_asset'], 
                                                       to_asset='USDT', conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), 
                               axis='columns')
    conversion_table['USDT_ask_price'] = \
        conversion_table.apply(lambda x: convert_price(size=x['ask_price'], from_asset=x['base_asset'], 
                                                       to_asset='USDT', conversion_table=conversion_table, 
                                                       exchange_info=exchange_info), 
                               axis='columns')
    conversion_table['USDT_bid_volume'] = \
        conversion_table['bid_volume'] * conversion_table['USDT_bid_price']
    conversion_table['USDT_ask_volume'] = \
        conversion_table['ask_volume'] * conversion_table['USDT_ask_price']

    conversion_table['USDT_price_change'] = \
        (conversion_table['USDT_price'] * conversion_table['USDT_open']) / \
        conversion_table['USDT_open']
    conversion_table['USDT_price_change_percent'] = \
        ((conversion_table['USDT_price_change'] / conversion_table['USDT_open']) * 100)

    conversion_table['is_shorted'] = False

    conversion_table_swapped = conversion_table.copy()
    conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 
                                     'weighted_average_price', 'close_shifted', 'close', 
                                     'last_volume', 'ask_price', 'ask_volume', 'bid_price', 
                                     'bid_volume', 'open', 'high', 'low', 'rolling_quote_volume', 
                                     'rolling_base_volume', 'open_time', 'close_time', 'first_ID', 
                                     'last_ID', 'count', 'quote_asset', 'base_asset', 
                                     'rolling_base_quote_volume', 'USDT_open', 'USDT_high', 
                                     'USDT_low', 'USDT_price', 'rolling_USDT_quote_volume', 
                                     'rolling_USDT_base_volume', 'USDT_ask_price', 'USDT_bid_price', 
                                     'USDT_ask_volume', 'USDT_bid_volume', 'USDT_price_change', 
                                     'USDT_price_change_percent', 'is_shorted']] = \
        conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 'weighted_average_price', 
                                         'close_shifted', 'close', 'last_volume', 'bid_price', 'bid_volume', 
                                         'ask_price', 'ask_volume', 'open', 'high', 'low', 'rolling_base_volume', 
                                         'rolling_quote_volume', 'open_time', 'close_time', 'first_ID', 'last_ID', 
                                         'count', 'base_asset', 'quote_asset', 'rolling_base_quote_volume', 
                                         'USDT_open', 'USDT_high', 'USDT_low', 'USDT_price', 
                                         'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                                         'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 
                                         'USDT_ask_volume', 'USDT_price_change', 'USDT_price_change_percent', 
                                         'is_shorted']].values
    conversion_table_swapped['symbol'] = \
        conversion_table_swapped['base_asset'] + conversion_table_swapped['quote_asset']
    conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 'ask_price', 
                                     'USDT_open', 'USDT_high', 'USDT_low', 'USDT_price', 'USDT_bid_price', 
                                     'USDT_ask_price', 'USDT_price_change', 'USDT_price_change_percent']] = \
        1 / conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 
                                             'bid_price', 'ask_price', 'USDT_open', 'USDT_high', 
                                             'USDT_low', 'USDT_price', 'USDT_bid_price', 'USDT_ask_price', 
                                             'USDT_price_change', 'USDT_price_change_percent']].astype(float)
    conversion_table_swapped['is_shorted'] = True

    conversion_table = concat([conversion_table, conversion_table_swapped], join='outer', axis='index')

    traded_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_volume = traded_volume['rolling_USDT_base_volume']
    conversion_table['rolling_traded_volume'] = \
        conversion_table.apply(lambda x: traded_volume.loc[x['base_asset']], axis='columns')
    traded_bid_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_bid_volume = traded_bid_volume['USDT_bid_volume']
    conversion_table['traded_bid_volume'] = \
        conversion_table.apply(lambda x: traded_bid_volume.loc[x['base_asset']], axis='columns')
    traded_ask_volume = conversion_table.groupby(by='base_asset').agg('sum')
    traded_ask_volume = traded_ask_volume['USDT_ask_volume']
    conversion_table['traded_ask_volume'] = \
        conversion_table.apply(lambda x: traded_ask_volume.loc[x['base_asset']], axis='columns')

    conversion_table['importance'] = \
        conversion_table['rolling_USDT_base_volume'] / conversion_table['rolling_traded_volume']

    conversion_table['importance_weighted_price'] = \
        conversion_table['USDT_price'] * conversion_table['importance']
    conversion_table['importance_weighted_bid_price'] = \
        conversion_table['USDT_bid_price'] * conversion_table['importance']
    conversion_table['importance_weighted_ask_price'] = \
        conversion_table['USDT_ask_price'] * conversion_table['importance']

    importance_weighted_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_price = importance_weighted_price['importance_weighted_price']
    conversion_table['traded_price'] = \
        conversion_table.apply(lambda x: importance_weighted_price.loc[x['base_asset']], axis='columns')
    importance_weighted_bid_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_bid_price = importance_weighted_bid_price['importance_weighted_bid_price']
    conversion_table['traded_bid_price'] = \
        conversion_table.apply(lambda x: importance_weighted_bid_price.loc[x['base_asset']], axis='columns')
    importance_weighted_ask_price = conversion_table.groupby(by='base_asset').agg('sum')
    importance_weighted_ask_price = importance_weighted_ask_price['importance_weighted_ask_price']
    conversion_table['traded_ask_price'] = \
        conversion_table.apply(lambda x: importance_weighted_ask_price.loc[x['base_asset']], axis='columns')

    conversion_table['bid_ask_change_percent'] = \
        ((conversion_table['ask_price'] - conversion_table['bid_price']) / conversion_table['ask_price'])
    conversion_table['bid_ask_volume_percent'] = \
        (conversion_table['bid_volume'] / (conversion_table['bid_volume'] + conversion_table['ask_volume']))
    conversion_table[['bid_ask_change_percent', 'bid_ask_volume_percent']] *= 100
    conversion_table['traded_bid_ask_change_percent'] = \
        ((conversion_table['traded_ask_price'] - conversion_table['traded_bid_price']) / \
         conversion_table['traded_ask_price'])
    conversion_table['traded_bid_ask_volume_percent'] = \
        (conversion_table['traded_bid_volume'] / (conversion_table['traded_bid_volume'] + \
                                                  conversion_table['traded_ask_volume']))
    conversion_table[['traded_bid_ask_change_percent', 'traded_bid_ask_volume_percent']] *= 100

    conversion_table = conversion_table[~conversion_table['is_shorted']]
    if as_pair:
        conversion_table = \
            conversion_table[['symbol', 'base_asset', 'quote_asset', 'is_shorted', 'price_change_percent', 
                              'weighted_average_price', 'open', 'high', 'low', 'close', 'close_shifted', 
                              'last_volume', 'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 
                              'close_time', 'last_ID', 'count', 'rolling_base_volume', 'rolling_quote_volume', 
                              'importance', 'USDT_price_change_percent', 'USDT_open', 
                              'USDT_high', 'USDT_low', 'USDT_price', 'rolling_USDT_base_volume', 
                              'rolling_USDT_quote_volume', 'USDT_bid_price', 'USDT_ask_price', 
                              'USDT_bid_volume', 'USDT_ask_volume', 'rolling_traded_volume', 
                              'traded_bid_volume', 'traded_ask_volume', 'traded_price', 
                              'traded_bid_price', 'traded_ask_price', 'bid_ask_change_percent', 
                              'bid_ask_volume_percent', 'traded_bid_ask_change_percent', 
                              'traded_bid_ask_volume_percent']]
    else:
        conversion_table = \
            conversion_table[['base_asset', 'USDT_price_change_percent', 'close_time', 'last_ID', 
                              'count', 'rolling_traded_volume', 'traded_bid_volume', 
                              'traded_ask_volume', 'traded_price', 'traded_bid_price', 
                              'traded_ask_price', 'traded_bid_ask_change_percent', 
                              'traded_bid_ask_volume_percent']]
        conversion_table['rolling_quote_volume'] = conversion_table['rolling_traded_volume'].copy()
        conversion_table = \
            conversion_table.rename(columns={'USDT_price_change_percent': 'price_change_percent', 
                                             'rolling_traded_volume': 'rolling_base_volume', 
                                             'traded_bid_volume': 'bid_volume', 
                                             'traded_ask_volume': 'ask_volume', 
                                             'traded_price': 'close', 
                                             'traded_bid_price': 'bid_price', 
                                             'traded_ask_price': 'ask_price', 
                                             'traded_bid_ask_change_percent': 'bid_ask_change_percent', 
                                             'traded_bid_ask_volume_percent': 'bid_ask_volume_percent'})
        conversion_table['symbol'] = conversion_table['base_asset'].copy()
        conversion_table['quote_asset'] = conversion_table['base_asset'].copy()
        df = conversion_table.groupby(by=['base_asset']).agg({'close_time': 'max', 
                                                              'last_ID': 'sum', 
                                                              'count': 'sum'})
        conversion_table.loc[:, ['close_time', 'last_ID', 'count']] = \
            conversion_table.apply(lambda x: df.loc[x['base_asset']], axis='columns')
        conversion_table = conversion_table.drop_duplicates(subset=['base_asset'], keep='first')
        conversion_table = conversion_table.reset_index(drop=True)

    conversion_table = conversion_table.sort_values(by='close_time')
    conversion_table = conversion_table.reset_index(drop=True)
    return conversion_table

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=False)
conversion_table.sort_values(by='rolling_base_volume', ascending=False).reset_index(drop=True).head(50)

In [ ]:
symbol = 'NEBL'

In [ ]:
import pandas as pd
crypto_output_log_1d = 'crypto_logs/crypto_output_log_1d.txt'
df_1d = pd.read_csv(crypto_output_log_1d, header=[0, 1], index_col=0)
df_1d.index = pd.DatetimeIndex(df_1d.index)
df_1d

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
df_15s[symbol]

In [ ]:
import pandas as pd
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min

In [ ]:
df_1min[symbol].tail(20)

In [ ]:
import pandas as pd
crypto_output_log_1h = 'crypto_logs/crypto_output_log_1h.txt'
df_1h = pd.read_csv(crypto_output_log_1h, header=[0, 1], index_col=0)
df_1h.index = pd.DatetimeIndex(df_1h.index)
df_1h

In [ ]:
df_1h[symbol].tail(20)

In [ ]:
df = df_1h.copy()
df

In [ ]:
df.tz_local

In [ ]:
import time
df = df_1h.copy()
is_dst = time.localtime().tm_isdst
timezone = time.tzname[is_dst]
offset_ns = time.altzone if is_dst else time.timezone
offset = (offset_ns / 60 / 60 * -1)
df.index = df.index + pd.Timedelta(-is_dst, unit='h')
df.index = df.index.tz_localize(tz=time.tzname[0])
df.index = pd.DatetimeIndex(df.index, ambiguous='infer')
df.index = df.index.tz_convert('UTC')
df.index

In [ ]:
df.index[-1].dst()

In [ ]:
import datetime
df.index.to_pydatetime()[-1]

In [ ]:
df = df_1min.copy()
df

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table
import pandas as pd

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)

conversion_table = df_1min.copy()
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table = conversion_table.drop(columns=['rolling_base_volume', 
                                                  'rolling_quote_volume'])
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table

In [ ]:
from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets

new_conversion_table = convert_ohlcvs_from_pairs_to_assets(conversion_table, 
                                                           exchange_info)
new_conversion_table

In [ ]:
new_conversion_table['ETH']